<a href="https://colab.research.google.com/github/rogertbarrow/DataCleaning/blob/main/Lifebear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install boto3

In [2]:
!pip install mysql-connector-python

In [3]:
import boto3
import psycopg2

import pandas as pd
import numpy as np
# from dotenv import load_env
import os
import csv
import re

import json
import random
from datetime import datetime
from dateutil import parser
import matplotlib.pyplot as plt
import math
import mysql.connector

In [4]:
file_path = "3.6M-Japan-lifebear.com-Largest-Notebook-App-UsersDB-csv-2019.csv"

lifebear_dataset = pd.read_csv(file_path, sep=';', low_memory=False)
lifebear_dataset

,id,login_id,mail_address,password,created_at,salt,birthday_on,gender
0,1,sugimoto,sugimoto@lifebear.co.jp,f0bac04aa1b45cf443d722d6f71c0250,2012-01-13 22:54:05,yGwBKynnsctI,1984-11-09,0.0
1,2,kou,nakanishi@lifebear.co.jp,48207c322ee5bb156ffec9f08c960aaa,2012-01-14 12:48:31,aha6EuRYCDvU,1986-11-13,0.0
2,3,yusuke,yuozawa1208@gmail.com,048261a8024ce51d379eb53cc51aaf33,2012-01-17 15:33:22,PVS59dPWk9BH,1984-12-08,0.0
3,4,entyan1106,endo1106@gmail.com,cd77a9dac26260a104facda5665eb3ab,2012-01-17 15:37:02,vLZI6TVCJowN,1987-11-06,0.0
4,5,kuriki,kuriki@wavy4.com,a026597c294cc48cd20ae361f10cbab1,2012-01-17 18:52:32,swFznWWk79fg,1986-10-21,0.0
...,...,...,...,...,...,...,...,...
3680437,11595925,avt,nakata5131@gmail.com,0261972b3e0449caf23085fa94beeb07,2019-02-21 16:08:13,GM4lB7bZPmFb,NaN,NaN
3680438,11595927,teru19730405,teru.teru.o-hira@ezweb.ne.jp,0979b1aaaea3b0809a9397e65f9f4e4e,2019-02-21 16:08:36,LEL9EO7SbTsk,NaN,NaN
3680439,11595928,akanen1231,n.akane.1231@icloud.com,9b06848ab38222e61e1a7993e848fc4e,2019-02-21 16:08:50,fRodU1FxjlTS,NaN,1.0
3680440,11595931,kukiau7,dansyakupoteto5103@yahoo.co.jp,238648b9a6f7ee3a2f7c231c0864a574,2019-02-21 16:10:07,g316d1ZLnYGv,NaN,NaN


In [5]:
print(lifebear_dataset.isnull().sum())

id                    0
login_id              2
mail_address          0
password              0
created_at            0
salt                  0
birthday_on     3294901
gender          1317523
dtype: int64


In [6]:
lifebear_dataset["login_id"] = lifebear_dataset["login_id"].fillna("NA")

In [7]:
lifebear_dataset["gender"] = lifebear_dataset["gender"].fillna(-1.0)

In [8]:
lifebear_dataset["birthday_on"] = lifebear_dataset["birthday_on"].fillna("NA")

In [9]:
lifebear_dataset.head(5)

,id,login_id,mail_address,password,created_at,salt,birthday_on,gender
0,1,sugimoto,sugimoto@lifebear.co.jp,f0bac04aa1b45cf443d722d6f71c0250,2012-01-13 22:54:05,yGwBKynnsctI,1984-11-09,0.0
1,2,kou,nakanishi@lifebear.co.jp,48207c322ee5bb156ffec9f08c960aaa,2012-01-14 12:48:31,aha6EuRYCDvU,1986-11-13,0.0
2,3,yusuke,yuozawa1208@gmail.com,048261a8024ce51d379eb53cc51aaf33,2012-01-17 15:33:22,PVS59dPWk9BH,1984-12-08,0.0
3,4,entyan1106,endo1106@gmail.com,cd77a9dac26260a104facda5665eb3ab,2012-01-17 15:37:02,vLZI6TVCJowN,1987-11-06,0.0
4,5,kuriki,kuriki@wavy4.com,a026597c294cc48cd20ae361f10cbab1,2012-01-17 18:52:32,swFznWWk79fg,1986-10-21,0.0


In [10]:
lifebear_dataset.tail(5)

,id,login_id,mail_address,password,created_at,salt,birthday_on,gender
3680437,11595925,avt,nakata5131@gmail.com,0261972b3e0449caf23085fa94beeb07,2019-02-21 16:08:13,GM4lB7bZPmFb,NA,-1.0
3680438,11595927,teru19730405,teru.teru.o-hira@ezweb.ne.jp,0979b1aaaea3b0809a9397e65f9f4e4e,2019-02-21 16:08:36,LEL9EO7SbTsk,NA,-1.0
3680439,11595928,akanen1231,n.akane.1231@icloud.com,9b06848ab38222e61e1a7993e848fc4e,2019-02-21 16:08:50,fRodU1FxjlTS,NA,1.0
3680440,11595931,kukiau7,dansyakupoteto5103@yahoo.co.jp,238648b9a6f7ee3a2f7c231c0864a574,2019-02-21 16:10:07,g316d1ZLnYGv,NA,-1.0
3680441,11595933,eiji4816,wdgbk492@yahoo.co.jp,7aab221dd8b37fbfce1f8868b2dbbc48,2019-02-21 16:10:47,i1oRgP6L8Nmy,NA,0.0


In [11]:
duplicates = lifebear_dataset[lifebear_dataset.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [id, login_id, mail_address, password, created_at, salt, birthday_on, gender]
Index: []


In [12]:
# prompt: How do I remove the time part from the "created_at" column?

# Assuming "created_at" column contains datetime values
lifebear_dataset['created_at'] = pd.to_datetime(lifebear_dataset['created_at']).dt.date


In [13]:
lifebear_dataset.head(5)

,id,login_id,mail_address,password,created_at,salt,birthday_on,gender
0,1,sugimoto,sugimoto@lifebear.co.jp,f0bac04aa1b45cf443d722d6f71c0250,2012-01-13,yGwBKynnsctI,1984-11-09,0.0
1,2,kou,nakanishi@lifebear.co.jp,48207c322ee5bb156ffec9f08c960aaa,2012-01-14,aha6EuRYCDvU,1986-11-13,0.0
2,3,yusuke,yuozawa1208@gmail.com,048261a8024ce51d379eb53cc51aaf33,2012-01-17,PVS59dPWk9BH,1984-12-08,0.0
3,4,entyan1106,endo1106@gmail.com,cd77a9dac26260a104facda5665eb3ab,2012-01-17,vLZI6TVCJowN,1987-11-06,0.0
4,5,kuriki,kuriki@wavy4.com,a026597c294cc48cd20ae361f10cbab1,2012-01-17,swFznWWk79fg,1986-10-21,0.0


In [14]:
# Rename a specific column
lifebear_dataset.rename(columns={'mail_address': 'e_mail_address','created_at': 'created_on','birthday_on': 'birthdate'}, inplace=True)

In [15]:
lifebear_dataset.head(5)

,id,login_id,e_mail_address,password,created_on,salt,birthdate,gender
0,1,sugimoto,sugimoto@lifebear.co.jp,f0bac04aa1b45cf443d722d6f71c0250,2012-01-13,yGwBKynnsctI,1984-11-09,0.0
1,2,kou,nakanishi@lifebear.co.jp,48207c322ee5bb156ffec9f08c960aaa,2012-01-14,aha6EuRYCDvU,1986-11-13,0.0
2,3,yusuke,yuozawa1208@gmail.com,048261a8024ce51d379eb53cc51aaf33,2012-01-17,PVS59dPWk9BH,1984-12-08,0.0
3,4,entyan1106,endo1106@gmail.com,cd77a9dac26260a104facda5665eb3ab,2012-01-17,vLZI6TVCJowN,1987-11-06,0.0
4,5,kuriki,kuriki@wavy4.com,a026597c294cc48cd20ae361f10cbab1,2012-01-17,swFznWWk79fg,1986-10-21,0.0


In [16]:
# prompt: Please check for invalid e-mail addresses, remove those rows, and export them to a separate .csv file.

def is_valid_email(email):
  """Check if an email address is valid using a regular expression."""
  pattern = r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
  return bool(re.match(pattern, email))

# Find rows with invalid email addresses
invalid_emails_df = lifebear_dataset[~lifebear_dataset['e_mail_address'].apply(is_valid_email)]

# Remove rows with invalid emails from the original DataFrame
lifebear_dataset = lifebear_dataset[lifebear_dataset['e_mail_address'].apply(is_valid_email)]

# Export the invalid email rows to a CSV file
invalid_emails_df.to_csv('invalid_email_addresses.csv', index=False)

print("Invalid email addresses have been exported to invalid_email_addresses.csv")


Invalid email addresses have been exported to invalid_email_addresses.csv


In [17]:
lifebear_dataset = lifebear_dataset.reset_index(drop=True)

In [18]:
lifebear_dataset.tail(5)

,id,login_id,e_mail_address,password,created_on,salt,birthdate,gender
3678734,11595925,avt,nakata5131@gmail.com,0261972b3e0449caf23085fa94beeb07,2019-02-21,GM4lB7bZPmFb,NA,-1.0
3678735,11595927,teru19730405,teru.teru.o-hira@ezweb.ne.jp,0979b1aaaea3b0809a9397e65f9f4e4e,2019-02-21,LEL9EO7SbTsk,NA,-1.0
3678736,11595928,akanen1231,n.akane.1231@icloud.com,9b06848ab38222e61e1a7993e848fc4e,2019-02-21,fRodU1FxjlTS,NA,1.0
3678737,11595931,kukiau7,dansyakupoteto5103@yahoo.co.jp,238648b9a6f7ee3a2f7c231c0864a574,2019-02-21,g316d1ZLnYGv,NA,-1.0
3678738,11595933,eiji4816,wdgbk492@yahoo.co.jp,7aab221dd8b37fbfce1f8868b2dbbc48,2019-02-21,i1oRgP6L8Nmy,NA,0.0


In [19]:
# Convert all text to lowercase
lifebear_dataset['login_id'] = lifebear_dataset['login_id'].str.lower()

# Strip leading and trailing whitespace
lifebear_dataset['login_id'] = lifebear_dataset['login_id'].str.strip()
lifebear_dataset['password'] = lifebear_dataset['password'].str.strip()
lifebear_dataset['salt'] = lifebear_dataset['salt'].str.strip()

In [20]:
# prompt: Output the cleaned dataframe to a .csv file.

lifebear_dataset.to_csv('cleaned_lifebear_dataset.csv', index=False)


In [21]:
# Specify the path to the folder containing the files
file_path = "cleaned_lifebear_dataset.csv"
output_folder = "sample_data//chunks"

def split_file(file_path, chunk_size, output_directory):
    os.makedirs(output_directory, exist_ok=True)

    with open(file_path, 'rb') as file:
        file_count = 0
        current_size = 0
        output_file = None

        for line in file:
            if output_file is None or current_size >= chunk_size * 1024 * 1024:
                if output_file:
                    output_file.close()
                file_count += 1
                current_size = 0
                file_name = f'lifebear_{str(file_count).zfill(3)}.csv'
                current_file_path = os.path.join(output_directory, file_name)
                output_file = open(current_file_path, 'w', encoding='utf-8')

            try:
                decoded_line = line.decode('utf-8')  # Try decoding with UTF-8
            except UnicodeDecodeError:
                decoded_line = line.decode('latin-1')  # If that fails, try decoding with Latin-1

            output_file.write(decoded_line)
            current_size += len(line)

        if output_file:
            output_file.close()

    print(f'Splitting complete. Total files created: {file_count}')

# Usage
split_file(file_path, 100, output_folder)

Splitting complete. Total files created: 4
